In [146]:
import json
from functools import reduce
import itertools
import re

In [6]:
def get_json_data():
    with open('templates/flare.json') as json_data:
        d = json.load(json_data)
    return d

In [26]:
data = get_json_data()

In [266]:
# Possible function to apply
def revenue_equation(data):
    return float(data['price']) * float(data['quantity'])

In [267]:
# Dictionary to store functions
dict_functions = {}

dict_functions['revenue_equation'] = revenue_equation

In [268]:
def get_value(node_name):
    return re.search(r':\d*', node_name).group()[1:]

def get_name_only(node_name):
    return re.search(r'\w*:', node_name).group()[:-1]

def get_children_dict(data, children_list):
    children_dict = {}
    for child in reduce(lambda d, k: d[k], children_list, data):
        children_dict[get_name_only(child['name'])] = get_value(child['name'])
    return children_dict

In [274]:
def rename(data, new_name, depth=0, node=[0]):
    
    if node:
        list1 = ['children'] * (len(node))
        list2 = node
    else:
        list1 = ['children'] * (depth + 1)
        list2 = range(depth + 1)
    
    # name value is a number of the function's name used to get its value
    name_value = re.search(r':\D*', new_name).group()[1:]

        
    dict_path = [x for x in itertools.chain.from_iterable(itertools.izip_longest(list1,list2)) if x!=None]
    
    print name_value
    if name_value:
        children_dict =  get_children_dict(data, dict_path+['children'])
        calculated_value = dict_functions[name_value](children_dict)
        new_name =  get_name_only(new_name) + ':' + str(calculated_value)
        print new_name
    
    reduce(lambda d, k: d[k], dict_path, data)['name'] = new_name
    return data

def view_name(data, depth=0):
    print reduce(lambda d, k: d[k], ['children', 0]* (depth+1), data)['name']

In [275]:
data = rename(data, 'revenue:revenue_equation')
data = rename(data, 'price:10', node = [0,0])
data = rename(data, 'quantity:5', node = [0,1])

revenue_equation
revenue:50.0




# Store JSON

In [276]:
with open('templates/myjson.json', 'w') as fp:
    json.dump(data, fp)